# 3 Nonlinear Equations and Complementarity Problems 

In [1]:

import numpy as np
from numpy import append, array, diagonal, tril, triu
from numpy.linalg import inv
from scipy.linalg import lu
#from scipy.linalg import solve
from pprint import pprint
from numpy import array, zeros, diag, diagflat, dot

from sympy import *
import sympy as sym
init_printing()

import matplotlib as mpl
# matplotlib for ploting
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D     # 3d
# for inline interactive plotting
%matplotlib notebook

#mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png', 'pdf')
# for better picture quality
%config InlineBackend.figure_format = 'retina'
#https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

import seaborn as sns
sns.set()
#sns.set_style(style= "whitegrid")
#plt.style.available
# bold graph style
plt.style.use('fivethirtyeight')

ModuleNotFoundError: No module named 'seaborn'

## 3.3 Newton's Method


In practice, most non-linear root-finding problems are solved using *Newton's method*
or one of its variants. Newton's method is based on the principle of *successive linearization*. Successive linearization calls for a hard non-linear problem to be replaced
with a sequence of simpler linear problems whose solutions converge to the solution
of the non-linear problem. Newton's method is typically formulated as a root-finding
technique, but may be used to solve a fixed-point problem $x = g(x)$ by recasting it
as the root-finding problem $f(x) = x - g(x) = 0$.


$$x:f(x)=0\,.$$

The method starts with a function $f$ defined over the real numbers $x$, the function's derivative $f ′$, and an initial guess $x_0$ for a root of the function $f$. If the function satisfies the assumptions made in the derivation of the formula and the initial guess is close, then a better approximation $x_1$ is

$$ x_{1}=x_{0}-{\frac {f(x_{0})}{f'(x_{0})}}\,.$$


Geometrically, $(x_1, 0)$ is the intersection of the x-axis and the tangent of the graph of f at $(x_0, f (x_0))$.

The process is repeated as

$$ x_{n+1}=x_{n}-{\frac {f(x_{n})}{f'(x_{n})}}\,$$
until a sufficiently accurate value is reached.

#### Derivation

Suppose $f : [a, b] → ℝ$ is a differentiable function defined on the interval $[a, b]$ with values in the real numbers $ℝ$. The formula for converging on the root can be easily derived. Suppose we have some current approximation $x_{n}$. Then we can derive the formula for a better approximation, $x_{n + 1}$ by referring to the diagram below. The equation of the tangent line to the curve $y = f (x)$ at the point $x = x_n$ is



$$ y=f'(x_{n})\,(x-x_{n})+f(x_{n}),$$


where $f′$ denotes the derivative of the function $f$.


The x-intercept of this line (the value of $x$ such that $y = 0$) is then used as the next approximation to the root, $x_{n+1}$. In other words, setting $y$ to zero and $x$ to $x_{n+1}$ gives

$$ 0=f'(x_{n})\,(x_{n+1}-x_{n})+f(x_{n}).$$
Solving for $x_{n+1}$ gives

$$ {\displaystyle x_{n+1}=x_{n}-{\frac {f(x_{n})}{f'(x_{n})}}.}$$



![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/NewtonIteration_Ani.gif/450px-NewtonIteration_Ani.gif)

ref: 
https://en.wikipedia.org/wiki/Newton%27s_method

In SciPy this algorithm is implemented by scipy.optimize.newton

Unlike bisection, the Newton-Raphson method uses local slope information

This is a double-edged sword:

- When the function is well-behaved, the Newton-Raphson method is faster than bisection
- When the function is less well-behaved, Newton-Raphson might fail


In [2]:
# Let's consider the same example we solved with bisection
from scipy.optimize import newton

f = lambda x: x**3 -2

newton(f, 0.2)   # Start the procedure with the initial guess x = 0.2

In [3]:
# Let's consider another simple example from the slides
from scipy.optimize import newton

f = lambda x: x**4 -2

newton(f, 2.3)   # Start the procedure with the initial guess x = 2.3

In [4]:
# Let's consider another simple example from the slides without using scipy

x = 2.3
for it in range(100):
    deltax = -(x**4.0 - 2.0)/(4.0*x**3.0)
    x = x + deltax
    if abs(deltax) < 1.e-10: #  accuracy
        break
        
print('The final iteration is:',it)
print('The computed root is:',x)  

The final iteration is: 7
The computed root is: 1.189207115002721


## Newton 

Signature: `newton(func, x0, fprime=None, args=(), tol=1.48e-08, maxiter=50, fprime2=None)`

Docstring:
Find a zero using the Newton-Raphson or secant method.

Find a zero of the function `func` given a nearby starting point `x0`.
The Newton-Raphson method is used if the derivative `fprime` of `func`
is provided, otherwise the secant method is used.  If the second order
derivative `fprime2` of `func` is provided, parabolic Halley's method
is used.


### Parameters


func : function

    The function whose zero is wanted. It must be a function of a
    **single variable** of the form f(x,a,b,c...), where a,b,c... are extra
    arguments that can be passed in the `args` parameter.
    
x0 : float

    An initial estimate of the zero that should be somewhere near the
    actual zero.
    
fprime : function, optional

    The derivative of the function when available and convenient. If it
    is None (default), then the secant method is used.



Since `newton` only takes one variable function, for multivariate problems we can use `fsolve` in scipy.optimize

### Example: Cournot Duopoly



To illustrate Newton's method in multiple dimensions, consider a simple Cournot duopoly model, in which the inverse demand for a good is 	$P(q_{1},q_{2})=\left( \dfrac{1}{q_{1}+q_{2}} \right)^{\epsilon}$





In [5]:
# Example from the textbook
q = np.array([1, 1]) # initial point/guess quantity
c = np.array([0.6, 0.8]) # cost vector
eta = 1.6 # elasticity 
e = -1/eta


In [6]:
# Cournot example. One update (first step) of newton algorithm 
fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q) # function value
fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+\
(e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c) # jacobian
q = q - np.linalg.inv(fjac).dot(fval) # newton update for the quantity
q

array([0.68765683, 0.55048639])

In [7]:
# first step result of objective value, and it should go to zero gradually
fval

array([-0.1542114, -0.3542114])

In [8]:
# first step result of jacobian
fjac

array([[-0.67598669,  0.12664449],
       [ 0.12664449, -0.87598669]])

### Newton's method

In [9]:
# Example from the textbook
# This executes a finite numer of Newton iterations
maxit = 150 # maximum number of iterations
tol = 10e-10 # tolerance level for convergence
# starting from [0.2 0.2] will cause a convergence problem (Newton's method has limitations)
#q = np.array([1, 1])
#c = np.array([0.6, 0.8])
#eta = 1.6
#e = -1/eta

for it in np.arange(maxit):

    fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q)
    fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+\
    (e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c)
    q = q - np.linalg.inv(fjac).dot(fval)
    if np.linalg.norm(fval)<tol:
        break
        
print('The final iteration is:',it)
print('The computed root is q:',q)
print('The function value is f:',fval) # it should be close to zero when a root is found

The final iteration is: 102
The computed root is q: [0.8395676  0.68879643]
The function value is f: [5.27553889e-10 6.43030407e-10]


Now let's write a function (mynewton) that achieves the same goal: solving a simple cournot model. It calls another function (cournot) that for the current guess computes the value of the FOC's for the two firms, and the related Jacobian.

In [10]:
def cournot(q):
    """
    A function for the Cournot game
    input: q quantity of production
    output: 
        fval: objective/function value,could be vector
        fjac: value of Jacobian of function, could be matrix
    """
    c = np.array([0.6, 0.8])
    eta = 1.6
    e = -1 / eta
    fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q)
    fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+\
            (e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c)
    return fval, fjac


In [11]:
# Newton 
def mynewton(f, x0, maxit=1000, tol=10e-10 ):
    """
    input: 
        f: function which output fval and fjac
        x0: initial guess of solution
        maxit: maximum number of iteration
        tol： tolerance
    output:
        x： solution        
    """
    x = x0
    for it in np.arange(maxit):
        fval, fjac = f(x)
        x = x - np.linalg.inv(fjac).dot(fval)
        if np.linalg.norm(fval)<tol:
            break
    return x    

In [12]:
mynewton(cournot, x0 = np.array([1, 1])) # it does converge to the same solution

array([0.8395676 , 0.68879643])

However, if we use a poor initial guess, the method might not work.
Without backstepping, an appropriate correction of the updates, the iterations do not converge with x0= np.array([0.2, 0.2]). The issue is that some updates are overstepping too far, entering an unstable region.



In [13]:
mynewton(cournot, x0 = np.array([0.2, 0.2])) # the procedure breaks down, and Python prompts an error message

/tmp/ipykernel_162/970703703.py:12: RuntimeWarning: invalid value encountered in scalar power
  fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q)
/tmp/ipykernel_162/970703703.py:13: RuntimeWarning: invalid value encountered in scalar power
  fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+\
/tmp/ipykernel_162/970703703.py:14: RuntimeWarning: invalid value encountered in scalar power
  (e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c)


array([nan, nan])

A more robust implementation can be found here
https://github.com/randall-romero/CompEcon-python/blob/master/compecon/nonlinear.py

### Python Libraries: the fsolve function

Python has a number of libraries designed to deal with root finding problems. Since scipy.optimize.newton only takes functions with one variable, for this application we can use fsolve in scipy.optimize to find the roots.

In [14]:
from scipy.optimize import fsolve

## Let's first check the library with a simple example 
def f(x):
    return x ** 3 - 2 * x ** 2

x = fsolve(f, 3)           # one root is at x=2.0

print("The root x is approximately x=%21.19g" % x)
print("The error is %g." % (2 - x))

The root x is approximately x= 2.000000000000006661
The error is -6.66134e-15.


/tmp/ipykernel_162/2743209447.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("The root x is approximately x=%21.19g" % x)
/tmp/ipykernel_162/2743209447.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("The error is %g." % (2 - x))



The return value of `fsolve` is a numpy array of length n for a root finding problem with `n` variables.

In [15]:
## Let's consider again the Cournot example
def cournotfval(q):
    """only output objective/function value"""
    c = np.array([0.6, 0.8])
    eta = 1.6
    e = -1/eta
    fval = np.sum(q)**e + e * np.sum(q)**(e-1)*(q) - np.diag(c).dot(q)
    return fval

def cournotjac(q):
    """only output value of jacobian of function """
    c = np.array([0.6, 0.8])
    eta = 1.6
    e = -1/eta
    fjac = e*np.sum(q)**(e-1)*(np.ones([2,2]))+e*np.sum(q)**(e-1)*(np.eye(2))+(e-1)*e*np.sum(q)**(e-2)*(q).dot(np.array([1, 1]))-np.diag(c)
    return fjac    



In [16]:
# Here the library function fsolve is using the analytic jacobian
# The procedure works even with the poor inital guess
q = fsolve(func = cournotfval,x0= np.array([0.2, 0.2]), fprime= cournotjac)  # with analytic jacobian, computed by cournotjac
print('The computed root is q:',q)

The computed root is q: [0.8395676  0.68879643]


In [18]:
# Here the library function fsolve is not using the analytic jacobian (it's replaced with a numerical approximation)
# The procedure works even with the poor inital guess
#q = fsolve(func = cournotfval, x0= np.array([1.0, 1.0]))
q = fsolve(func = cournotfval, x0= np.array([0.2, 0.2]))
print('The computed root is q:',q)

The computed root is q: [0.8395676  0.68879643]
